## Zeroshot Topic Models (ZTMs) 

*   List item
*   List item



Using ZTMs to produce topic models
 

## Preprocessing

to ensure the citations are stripped of html and preprocessed according to a pipeline 

In [1]:
!pip install contextualized_topic_models pyLDAvis scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz (15.9 MB)


In [2]:
from bs4 import BeautifulSoup
import string
import re

def remove_html(x):
    soup = BeautifulSoup(x, 'html.parser')
    tags = soup.find_all('cite')
    text = soup.get_text()
    text = text.replace('  ', '')
    text = text.replace(' et ', '')
    text = text.replace(' al ', '')
    text = text.replace(' et. ', '')
    text = text.replace(' al. ', '')
    return text

In [3]:
import scispacy
import spacy
import string

nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe("merge_entities")

def merge_entities(x):
    doc = nlp(x)
    return " ".join([re.sub(r'\W+', '', t.text.strip().translate(str.maketrans('', '', string.punctuation)).replace(" ", "_").lower()) for t in doc])


In [4]:
import pandas as pd

citations = pd.read_csv('./example_doc_citations.csv')
citations['text'] = citations['text'].apply(
    lambda x : remove_html(x)
)
citations['ner_merged_text'] = citations['text'].apply(
    lambda x: merge_entities(x)
)

In [5]:
from gensim.corpora.dictionary import Dictionary

removal = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
remove_text = ['al', 'et', 'al.']
authors = [
    'mongeon',
    'hus'
]


unpreprocessed_corpus = citations['text']

preprocessed_documents = []
for doc in nlp.pipe(citations['ner_merged_text']):
    proj_tok = [token.lemma_.lower() for token in doc if "_" in token.text or (len(token.text) > 2 and token.text not in remove_text and token.pos_ not in removal and not token.is_stop and token.is_alpha)]
    proj_tok = [
        tok for tok in proj_tok
        if all([author not in tok for author in authors])
    ]
    preprocessed_documents.append(' '.join(proj_tok))
    
    
texts = [doc.split(' ') for doc in preprocessed_documents]
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=2, no_above=0.9)


In [6]:
preprocessed_documents = [' '.join([dictionary[token[0]].replace("-", "_").replace("®", "_") for token in dictionary.doc2bow(text)]) for text in texts]

In [7]:
from contextualized_topic_models.models.ctm import ZeroShotTM, CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk
import torch
import random
import numpy as np

In [8]:
def fix_seeds():
    torch.manual_seed(10)
    torch.cuda.manual_seed(10)
    np.random.seed(10)
    random.seed(10)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

In [9]:
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO

fix_seeds()

num_topics = [3, 5, 10, 15, 20, 25, 50, 100, 250]

corpus = [doc.split(' ') for doc in preprocessed_documents]
model_results = []
embedding_models = [
    "paraphrase-distilroberta-base-v2",
    "sentence-transformers/allenai-specter",
    "allenai/aspire-sentence-embedder",
    "allenai/aspire-contextualsentence-multim-compsci", 
]
for embedding_model in embedding_models:
    tp = TopicModelDataPreparation(embedding_model)
    training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)
    for n_components in num_topics:
        print("num topics:", n_components)
        ztm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, 
                     n_components=n_components, num_epochs=50)
        ztm.fit(training_dataset, n_samples=20)
        coh = CoherenceNPMI(ztm.get_topic_lists(10), [doc.split(' ') for doc in preprocessed_documents])
        coh_score = coh.score()
        print("coherence score:", coh_score)
        diversity_score = InvertedRBO(ztm.get_topic_lists(10)).score()
        print("diversity score LDA:", diversity_score)
        model_results.append({
            "coherence": coh_score,
            "diversity": diversity_score,
            "num_topics": n_components,
            "model": ztm,
            "embedding_model": embedding_model,
            "tp": tp,
            "training_dataset": training_dataset
        })

/usr/local/lib/python3.7/dist-packages/contextualized_topic_models/utils/data_preparation.py:57: UserWarning: the longest document in your collection has 178 words, the model instead truncates to 128 tokens.
  warnings.warn(f"the longest document in your collection has {max_local_length} words, the model instead "


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


num topics: 3


Epoch: [50/50]	 Seen Samples: [39550/39550]	Train Loss: 173.12433085097976	Time: 0:00:00.353339: : 50it [00:17,  2.79it/s]
Sampling: [20/20]: : 20it [00:06,  3.12it/s]


...

coherence score: -0.09411504321138661
diversity score LDA: 0.9381236065526647
num topics: 100


Epoch: [50/50]	 Seen Samples: [39550/39550]	Train Loss: 217.62912971120417	Time: 0:00:00.429246: : 50it [00:21,  2.29it/s]
Sampling: [20/20]: : 20it [00:08,  2.47it/s]


coherence score: -0.17887581623239343
diversity score LDA: 0.9499786003006969
num topics: 250


Epoch: [50/50]	 Seen Samples: [39550/39550]	Train Loss: 282.9947782919169	Time: 0:00:00.439273: : 50it [00:21,  2.28it/s]
Sampling: [20/20]: : 20it [00:08,  2.29it/s]


coherence score: -0.31226021816547955
diversity score LDA: 0.9776125841885076


## Select Top Topic Model

In [10]:
results_df = pd.DataFrame(model_results)

In [11]:
results_df.sort_values(by='coherence', ascending=False)[
    ['coherence','diversity','num_topics', 'embedding_model']
]

,coherence,diversity,num_topics,embedding_model
12,-0.041906,0.935548,15,sentence-transformers/allenai-specter
23,-0.044553,0.928589,25,allenai/aspire-sentence-embedder
6,-0.044599,0.925388,50,paraphrase-distilroberta-base-v2
31,-0.055803,0.925411,20,allenai/aspire-contextualsentence-multim-compsci
21,-0.059224,0.935017,15,allenai/aspire-sentence-embedder
4,-0.062100,0.941541,20,paraphrase-distilroberta-base-v2
15,-0.065892,0.922158,50,sentence-transformers/allenai-specter
22,-0.067374,0.935290,20,allenai/aspire-sentence-embedder
14,-0.071861,0.925275,25,sentence-transformers/allenai-specter
5,-0.074043,0.953038,25,paraphrase-distilroberta-base-v2


In [12]:
results_df.sort_values(by='diversity', ascending=False)[
    ['coherence','diversity','num_topics', 'embedding_model']
]

,coherence,diversity,num_topics,embedding_model
0,-0.294258,1.000000,3,paraphrase-distilroberta-base-v2
19,-0.138900,1.000000,5,allenai/aspire-sentence-embedder
27,-0.104213,1.000000,3,allenai/aspire-contextualsentence-multim-compsci
28,-0.091580,0.995050,5,allenai/aspire-contextualsentence-multim-compsci
18,-0.162781,0.983499,3,allenai/aspire-sentence-embedder
9,-0.181935,0.980968,3,sentence-transformers/allenai-specter
35,-0.312260,0.977613,250,allenai/aspire-contextualsentence-multim-compsci
10,-0.150384,0.976994,5,sentence-transformers/allenai-specter
26,-0.286995,0.971587,250,allenai/aspire-sentence-embedder
1,-0.118976,0.965240,5,paraphrase-distilroberta-base-v2


In [17]:
ztm_model = results_df.iloc[27]['model']

In [18]:
ztm_model.get_topic_lists(5)

[['datum', 'observe', 'pubme', 'issue', 'english_language'],
 ['literature', 'scopus_index', 'conduct', 'obtain', 'abstract'],
 ['scopus', 'large', 'database', 'journal', 'wos']]

## Visualize topics

In [15]:
import pyLDAvis as vis

ztm = results_df.sort_values(by='coherence', ascending=False).iloc[0]['model']
tp = results_df.sort_values(by='coherence', ascending=False).iloc[0]['tp']
training_dataset = results_df.sort_values(by='coherence', ascending=False).iloc[0]['training_dataset']
lda_vis_data = ztm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=1)

ztm_pd = vis.prepare(**lda_vis_data)
vis.display(ztm_pd)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
Sampling: [1/1]: : 1it [00:00,  2.37it/s]
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


## Compute Topic Index

In [16]:
import numpy as np
import json
for i, result in results_df.iterrows():
    documents = []
    topic_predictions = result['model'].get_thetas(training_dataset, n_samples=5)
    for topics, doi, cite_id in zip(topic_predictions, citations['source_doi'], citations['id']):
        topic_number = np.argmax(topics)
        keywords = [term for term in result['model'].get_topic_lists(10)[topic_number]]
        documents.append({
            "doi": doi,
            "cite_id": cite_id,
            "keywords": [word for word in set(keywords)]
        })
        with open(f'./ztm_{result["num_topics"]}_{result["embedding_model"].replace("/", "_")}_topic_index.json', 'w+') as f:
            json.dump({
                "embedding_model": result["embedding_model"],
                "topics": result["num_topics"],
                "diversity": result["diversity"],
                "coherence": result["coherence"],
                "documents": documents
            }, f)

Sampling: [5/5]: : 5it [00:01,  2.51it/s]
Sampling: [5/5]: : 5it [00:01,  2.52it/s]
Sampling: [5/5]: : 5it [00:01,  2.51it/s]
Sampling: [5/5]: : 5it [00:01,  2.55it/s]
Sampling: [5/5]: : 5it [00:01,  2.53it/s]
Sampling: [5/5]: : 5it [00:01,  2.52it/s]
Sampling: [5/5]: : 5it [00:02,  2.46it/s]
Sampling: [5/5]: : 5it [00:02,  2.44it/s]
Sampling: [5/5]: : 5it [00:02,  2.37it/s]
Sampling: [5/5]: : 5it [00:01,  2.51it/s]
Sampling: [5/5]: : 5it [00:01,  2.53it/s]
Sampling: [5/5]: : 5it [00:02,  2.49it/s]
Sampling: [5/5]: : 5it [00:01,  2.52it/s]
Sampling: [5/5]: : 5it [00:02,  2.47it/s]
Sampling: [5/5]: : 5it [00:01,  2.52it/s]
Sampling: [5/5]: : 5it [00:02,  2.46it/s]
Sampling: [5/5]: : 5it [00:02,  2.45it/s]
Sampling: [5/5]: : 5it [00:02,  2.33it/s]
Sampling: [5/5]: : 5it [00:02,  2.48it/s]
Sampling: [5/5]: : 5it [00:02,  2.46it/s]
Sampling: [5/5]: : 5it [00:02,  2.47it/s]
Sampling: [5/5]: : 5it [00:02,  2.44it/s]
Sampling: [5/5]: : 5it [00:02,  2.45it/s]
Sampling: [5/5]: : 5it [00:02,  2.